In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [2]:
import senteval
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, models
from glob import glob
from tqdm import tqdm

In [3]:
# SentEval prepare and batcher
def prepare(params, samples):
    return

def batcher(params, batch):
    batch = [' '.join(sent) if sent != [] else '.' for sent in batch]
    embeddings = params['encoder'](batch)
    return embeddings

# Parameter check

In [3]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: 
            continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
#     print(table)
    print(f"Total Trainable Params: {total_params:,}")
    return total_params

pool_mode='cls'
mlp_mode=True
model_list = glob(f'output/Simcse_original_*{pool_mode}*MLP{mlp_mode}*')
for m in model_list[:]:
    print(f"Model:{m}")
    sim_cse = SentenceTransformer(m)
    
    count_parameters(sim_cse)

Model:output/Simcse_original_BERT_original_cls_MLPTrue
Total Trainable Params: 109,482,240
Model:output/Simcse_original_nreimers_albert_small_v2_cls_MLPTrue
Total Trainable Params: 12,274,176
Model:output/Simcse_original_nreimers_BERT_Mini_L_4_H_256_A_4_cls_MLPTrue
Total Trainable Params: 11,236,352
Model:output/Simcse_original_nreimers_MiniLM_L6_H384_uncased_cls_MLPTrue
Total Trainable Params: 22,861,056
Model:output/Simcse_original_bert_large_uncased_cls_MLPTrue
Total Trainable Params: 336,191,488
Model:output/Simcse_original_distilbert_base_cased_cls_MLPTrue
Total Trainable Params: 65,781,504
Model:output/Simcse_original_nreimers_TinyBERT_L_4_H_312_v2_cls_MLPTrue
Total Trainable Params: 14,447,904
Model:output/Simcse_original_roberta_base_cls_MLPTrue
Total Trainable Params: 125,236,224
Model:output/Simcse_original_nreimers_BERT_Tiny_L_2_H_128_A_2_cls_MLPTrue
Total Trainable Params: 4,402,432
Model:output/Simcse_original_roberta_large_cls_MLPTrue
Total Trainable Params: 356,409,344
M

In [14]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: 
            continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
#     print(table)
    print(f"Total Trainable Params: {total_params:,}")
    return total_params

pool_mode='cls'
mlp_mode=True
model_list = ['microsoft/Multilingual-MiniLM-L12-H384','distilbert-base-multilingual-cased','bert-base-multilingual-cased','xlm-roberta-base']
for m in model_list[:]:
    print(f"Model:{m}")
    sim_cse = SentenceTransformer(m)
    
    count_parameters(sim_cse)

Model:microsoft/Multilingual-MiniLM-L12-H384


Total Trainable Params: 117,653,760
Model:distilbert-base-multilingual-cased


Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/distilbert-base-multilingual-cased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total Trainable Params: 134,734,080
Model:bert-base-multilingual-cased


Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total Trainable Params: 177,853,440
Model:xlm-roberta-base


Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total Trainable Params: 278,043,648


# ConGen 

we show only 1 random seed, if you want 3 random seeds, please see our github link (attached in the paper)

In [4]:
def sorting_score(avg_score,file_name, score_list, name_list):
    for idx,_ in enumerate(score_list):
        if avg_score > score_list[idx]:
            score_list.insert(idx,avg_score)
            name_list.insert(idx,file_name)
            break
    return score_list, name_list

In [7]:
model_list = glob(f'congen_models/Asym*')

all_scores = [0] * len(model_list)
name_files = []
for model in tqdm(model_list[:]):
    print(f"Model:{model}")
    sim_cse = SentenceTransformer(model)

    params = {'task_path': 'SentEval/data/', 'usepytorch': True, 'kfold': 10}
    params['encoder'] = sim_cse.encode
    se = senteval.engine.SE(params, batcher, prepare)

    transfer_tasks = ['STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'STSBenchmark', 'SICKRelatedness']
    results = se.eval(transfer_tasks)

    spearman_val = 0
    text = []
    for key in results.keys():
        text.append(key)
        if key not in ['STSBenchmark','SICKRelatedness']:
            result_temp = results[key]['all']['spearman']['all']
            spearman_val+=result_temp
            text.append(f"Spearman:{result_temp*100:.2f}")
        else:
            result_temp = results[key]['test']['spearman'].correlation
            spearman_val+=result_temp
            text.append(f"Spearman:{result_temp*100:.2f}")
    avg_score = round(spearman_val/len(results.keys())*100,2)
    all_scores,name_files = sorting_score(avg_score,model,all_scores,name_files)
    print(f"Avg:{avg_score}")
    print(f"*"*50)



  0%|          | 0/10 [00:00<?, ?it/s]

Model:congen_models/Asym_BERT-Small


/workspace/m_sentence_embedding/SimCSE/SentEval/senteval/sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
/workspace/m_sentence_embedding/SimCSE/SentEval/senteval/sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]
 10%|█         | 1/10 [01:02<09:24, 62.67s/it]

Avg:78.83
**************************************************
Model:congen_models/Asym_MiniLM-L3


 20%|██        | 2/10 [01:43<07:27, 55.99s/it]

...

Avg:80.09
**************************************************
Model:congen_models/Asym_BERT-Tiny


 90%|█████████ | 9/10 [09:59<01:07, 67.39s/it]

Avg:77.12
**************************************************
Model:congen_models/Asym_MiniLM-L6


100%|██████████| 10/10 [10:47<00:00, 64.79s/it]

Avg:79.2
**************************************************


# L2 Mono T = SimCSE(Wiki) S = small model

In [10]:
model_list = glob(f'output/l2_mono/L2-no-aug-*')
for model in model_list:
    try:
        word_embedding_model = models.Transformer(model, max_seq_length=128)
        pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
        sim_cse = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    except:
        sim_cse = SentenceTransformer(model)
    
    params = {'task_path': 'SentEval/data/', 'usepytorch': True, 'kfold': 10}
    params['encoder'] = sim_cse.encode
    se = senteval.engine.SE(params, batcher, prepare)

    transfer_tasks = ['STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'STSBenchmark', 'SICKRelatedness']
    results = se.eval(transfer_tasks)

    print(f"Model:{model}")
    spearman_val = 0
    for key in results.keys():
        print(key)
    #     print(results)
        if key not in ['STSBenchmark','SICKRelatedness']:
            result_temp = results[key]['all']['spearman']['all']
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
        else:
            result_temp = results[key]['test']['spearman'].correlation
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
    print(f"Avg:{(spearman_val/len(results.keys()))*100:.2f}")
    print(f"*"*50)
    print()

Model:output/l2_mono/L2-no-aug-T-princeton-nlp-unsup-simcse-roberta-large-S-bert-base-uncased
STS12
Spearman:74.07
STS13
Spearman:85.69
STS14
Spearman:77.37
STS15
Spearman:85.37
STS16
Spearman:80.38
STSBenchmark
Spearman:81.80
SICKRelatedness
Spearman:69.59
Avg:79.18
**************************************************

Model:output/l2_mono/L2-no-aug-T-princeton-nlp-unsup-simcse-roberta-large-S-nreimers-MiniLM-L3-H384-uncased
STS12
Spearman:70.68
STS13
Spearman:82.39
STS14
Spearman:74.80
STS15
Spearman:82.70
STS16
Spearman:76.91
STSBenchmark
Spearman:74.71
SICKRelatedness
Spearman:65.24
Avg:75.35
**************************************************

Model:output/l2_mono/L2-no-aug-T-princeton-nlp-unsup-simcse-roberta-large-S-nreimers-BERT-Tiny_L-2_H-128_A-2
STS12
Spearman:68.74
STS13
Spearman:78.81
STS14
Spearman:71.12
STS15
Spearman:78.80
STS16
Spearman:72.00
STSBenchmark
Spearman:67.92
SICKRelatedness
Spearman:61.55
Avg:71.28
**************************************************

Model:outpu

# SKD distill T=Robert-Large (SimCSE-wiki1M)

In [7]:
model_list = glob(f'output/skd/skd_mono-seed-T*')
for model in model_list:
    sim_cse = SentenceTransformer(model)
    
    params = {'task_path': 'SentEval/data/', 'usepytorch': True, 'kfold': 10}
    params['encoder'] = sim_cse.encode
    se = senteval.engine.SE(params, batcher, prepare)

    transfer_tasks = ['STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'STSBenchmark', 'SICKRelatedness']
    results = se.eval(transfer_tasks)

    print(f"Model:{model}")
    spearman_val = 0
    for key in results.keys():
        print(key)
    #     print(results)
        if key not in ['STSBenchmark','SICKRelatedness']:
            result_temp = results[key]['all']['spearman']['all']
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
        else:
            result_temp = results[key]['test']['spearman'].correlation
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
    print(f"Avg:{(spearman_val/len(results.keys()))*100:.2f}")
    print(f"*"*50)
    print()

Model:output/skd/skd_mono-seed-T-princeton-nlp-unsup-simcse-roberta-large-S-nreimers-MiniLM-L3-H384-uncased
STS12
Spearman:70.06
STS13
Spearman:78.67
STS14
Spearman:69.50
STS15
Spearman:78.75
STS16
Spearman:71.73
STSBenchmark
Spearman:73.45
SICKRelatedness
Spearman:67.64
Avg:72.83
**************************************************

Model:output/skd/skd_mono-seed-T-princeton-nlp-unsup-simcse-roberta-large-S-nreimers-TinyBERT_L-6_H-768_v2
STS12
Spearman:74.65
STS13
Spearman:82.83
STS14
Spearman:75.20
STS15
Spearman:83.24
STS16
Spearman:78.44
STSBenchmark
Spearman:78.79
SICKRelatedness
Spearman:70.78
Avg:77.70
**************************************************

Model:output/skd/skd_mono-seed-T-princeton-nlp-unsup-simcse-roberta-large-S-bert-base-uncased
STS12
Spearman:75.18
STS13
Spearman:84.36
STS14
Spearman:75.77
STS15
Spearman:82.34
STS16
Spearman:78.96
STSBenchmark
Spearman:80.30
SICKRelatedness
Spearman:71.00
Avg:78.27
**************************************************

Model:output/

# Making distill T=Robert-Large (SimCSE-wiki1M)

In [ ]:
model_list = glob(f'output/making/making_mono-T*')

for model in model_list:
    sim_cse = SentenceTransformer(model)
    
    params = {'task_path': 'SentEval/data/', 'usepytorch': True, 'kfold': 10}
    params['encoder'] = sim_cse.encode
    se = senteval.engine.SE(params, batcher, prepare)

    transfer_tasks = ['STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'STSBenchmark', 'SICKRelatedness']
    results = se.eval(transfer_tasks)

    print(f"Model:{model}")
    spearman_val = 0
    for key in results.keys():
        print(key)
    #     print(results)
        if key not in ['STSBenchmark','SICKRelatedness']:
            result_temp = results[key]['all']['spearman']['all']
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
        else:
            result_temp = results[key]['test']['spearman'].correlation
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
    print(f"Avg:{(spearman_val/len(results.keys()))*100:.2f}")
    print(f"*"*50)
    print()

/workspace/m_sentence_embedding/SBERT/SentEval/senteval/sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
/workspace/m_sentence_embedding/SBERT/SentEval/senteval/sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


Model:output/making/making_mono-T-princeton-nlp-unsup-simcse-roberta-large-S-nreimers-BERT-Small-L-4_H-512_A-8
STS12
Spearman:71.05
STS13
Spearman:81.65
STS14
Spearman:74.04
STS15
Spearman:82.49
STS16
Spearman:75.66
STSBenchmark
Spearman:77.85
SICKRelatedness
Spearman:69.35
Avg:76.01
**************************************************

Model:output/making/making_mono-T-princeton-nlp-unsup-simcse-roberta-large-S-nreimers-MiniLM-L6-H384-uncased
STS12
Spearman:68.08
STS13
Spearman:81.22
STS14
Spearman:72.57
STS15
Spearman:81.13
STS16
Spearman:75.14
STSBenchmark
Spearman:76.11
SICKRelatedness
Spearman:68.93
Avg:74.74
**************************************************

Model:output/making/making_mono-T-princeton-nlp-unsup-simcse-roberta-large-S-nreimers-BERT-Mini_L-4_H-256_A-4
STS12
Spearman:68.12
STS13
Spearman:78.27
STS14
Spearman:69.43
STS15
Spearman:78.94
STS16
Spearman:71.76
STSBenchmark
Spearman:73.39
SICKRelatedness
Spearman:67.44
Avg:72.48
********************************************

# BSL original

In [4]:
pool_mode = 'cls'
mlp_mode = True
model_list = glob(f'output/bsl_original/seed-BSL_unsup_tuning-*')
for model in model_list:
    sim_cse = SentenceTransformer(model)
    print(f"Model:{model}")
#     word_embedding_model = models.Transformer(model, max_seq_length=32)
#     pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),pooling_mode=pool_mode)
#     sim_cse = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    
    params = {'task_path': 'SentEval/data/', 'usepytorch': True, 'kfold': 10}
    params['encoder'] = sim_cse.encode
    se = senteval.engine.SE(params, batcher, prepare)

    transfer_tasks = ['STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'STSBenchmark', 'SICKRelatedness']
    results = se.eval(transfer_tasks)

    
    spearman_val = 0
    for key in results.keys():
        print(key)
    #     print(results)
        if key not in ['STSBenchmark','SICKRelatedness']:
            result_temp = results[key]['all']['spearman']['all']
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
        else:
            result_temp = results[key]['test']['spearman'].correlation
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
    print(f"Avg:{(spearman_val/len(results.keys()))*100:.2f}")
    print(f"*"*50)
    print()

Model:output/bsl_original/seed-BSL_unsup_tuning-nreimers-BERT-Small-L-4_H-512_A-8


/workspace/m_sentence_embedding/SimCSE/SentEval/senteval/sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
/workspace/m_sentence_embedding/SimCSE/SentEval/senteval/sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


STS12
Spearman:70.17
STS13
Spearman:65.73
STS14
Spearman:66.87
STS15
Spearman:80.27
STS16
Spearman:74.53
STSBenchmark
Spearman:74.26
SICKRelatedness
Spearman:65.45
Avg:71.04
**************************************************

Model:output/bsl_original/seed-BSL_unsup_tuning-nreimers-TinyBERT_L-6_H-768_v2
STS12
Spearman:70.23
STS13
Spearman:67.07
STS14
Spearman:66.61
STS15
Spearman:80.43
STS16
Spearman:75.29
STSBenchmark
Spearman:75.53
SICKRelatedness
Spearman:68.81
Avg:72.00
**************************************************

Model:output/bsl_original/seed-BSL_unsup_tuning-nreimers-TinyBERT_L-4_H-312_v2
STS12
Spearman:67.76
STS13
Spearman:58.93
STS14
Spearman:60.21
STS15
Spearman:76.78
STS16
Spearman:69.30
STSBenchmark
Spearman:69.71
SICKRelatedness
Spearman:66.44
Avg:67.02
**************************************************

Model:output/bsl_original/seed-BSL_unsup_tuning-microsoft-MiniLM-L12-H384-uncased
STS12
Spearman:63.16
STS13
Spearman:58.91
STS14
Spearman:54.38
STS15
Spearman:72.3

# SimCSE (Supervised)

In [24]:
pool_mode = 'cls'
mlp_mode = True
model_list = glob(f'output/simcse_original/Simcse_supervised_seed_*')
for model in model_list:
    sim_cse = SentenceTransformer(model)
    print(f"Model:{model}")
#     word_embedding_model = models.Transformer(model, max_seq_length=32)
#     pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),pooling_mode=pool_mode)
#     sim_cse = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    
    params = {'task_path': 'SentEval/data/', 'usepytorch': True, 'kfold': 10}
    params['encoder'] = sim_cse.encode
    se = senteval.engine.SE(params, batcher, prepare)

    transfer_tasks = ['STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'STSBenchmark', 'SICKRelatedness']
    results = se.eval(transfer_tasks)

    
    spearman_val = 0
    for key in results.keys():
        print(key)
    #     print(results)
        if key not in ['STSBenchmark','SICKRelatedness']:
            result_temp = results[key]['all']['spearman']['all']
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
        else:
            result_temp = results[key]['test']['spearman'].correlation
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
    print(f"Avg:{(spearman_val/len(results.keys()))*100:.2f}")
    print(f"*"*50)
    print()

Model:output/simcse_original/Simcse_supervised_seed_nreimers_BERT_Tiny_L_2_H_128_A_2_cls_MLPTrue
STS12
Spearman:67.76
STS13
Spearman:73.88
STS14
Spearman:67.58
STS15
Spearman:77.23
STS16
Spearman:72.29
STSBenchmark
Spearman:73.80
SICKRelatedness
Spearman:73.91
Avg:72.35
**************************************************

Model:output/simcse_original/Simcse_supervised_seed_nreimers_BERT_Mini_L_4_H_256_A_4_cls_MLPTrue
STS12
Spearman:72.68
STS13
Spearman:76.34
STS14
Spearman:73.01
STS15
Spearman:81.37
STS16
Spearman:76.58
STSBenchmark
Spearman:78.33
SICKRelatedness
Spearman:77.34
Avg:76.52
**************************************************

Model:output/simcse_original/Simcse_supervised_seed_nreimers_BERT_Small_L_4_H_512_A_8_cls_MLPTrue
STS12
Spearman:73.51
STS13
Spearman:78.70
STS14
Spearman:76.51
STS15
Spearman:83.10
STS16
Spearman:78.65
STSBenchmark
Spearman:81.29
SICKRelatedness
Spearman:78.37
Avg:78.59
**************************************************

Model:output/simcse_original/S

# SimCSE (Wiki1M) Without MLP

In [9]:
model_list = glob(f'output/simcse_original/Simcse_original_seed_*wikiTrue_nliFalse')
# model_list = ['output/Simcse_original_bert_base_uncased_cls_MLPTrue']
# model_list = glob(f'output/Simcse_original_*large*{pool_mode}*MLP{mlp_mode}*')
for model in model_list:
#     sim_cse = SentenceTransformer(model)
    word_embedding_model = models.Transformer(model, max_seq_length=32)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),pooling_mode=pool_mode)
    sim_cse = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    params = {'task_path': 'SentEval/data/', 'usepytorch': True, 'kfold': 10}
    params['encoder'] = sim_cse.encode
    se = senteval.engine.SE(params, batcher, prepare)

    transfer_tasks = ['STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'STSBenchmark', 'SICKRelatedness']
    results = se.eval(transfer_tasks)

    print(f"Model:{model}")
    spearman_val = 0
    for key in results.keys():
        print(key)
    #     print(results)
        if key not in ['STSBenchmark','SICKRelatedness']:
            result_temp = results[key]['all']['spearman']['all']
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
        else:
            result_temp = results[key]['test']['spearman'].correlation
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
    print(f"Avg:{(spearman_val/len(results.keys()))*100:.2f}")
    print(f"*"*50)
    print()

/workspace/m_sentence_embedding/SimCSE/SentEval/senteval/sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
/workspace/m_sentence_embedding/SimCSE/SentEval/senteval/sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


Model:output/simcse_original/Simcse_original_seed_2_nreimers_MiniLM_L6_H384_uncased_cls_MLPTrue_wikiTrue_nliFalse
STS12
Spearman:46.08
STS13
Spearman:58.48
STS14
Spearman:44.47
STS15
Spearman:57.90
STS16
Spearman:61.82
STSBenchmark
Spearman:50.80
SICKRelatedness
Spearman:55.75
Avg:53.61
**************************************************

Model:output/simcse_original/Simcse_original_seed_2_nreimers_BERT_Tiny_L_2_H_128_A_2_cls_MLPTrue_wikiTrue_nliFalse
STS12
Spearman:60.97
STS13
Spearman:69.92
STS14
Spearman:59.90
STS15
Spearman:71.39
STS16
Spearman:66.23
STSBenchmark
Spearman:60.22
SICKRelatedness
Spearman:60.06
Avg:64.10
**************************************************

Model:output/simcse_original/Simcse_original_seed_2_nreimers_TinyBERT_L_4_H_312_v2_cls_MLPTrue_wikiTrue_nliFalse
STS12
Spearman:60.31
STS13
Spearman:66.74
STS14
Spearman:57.42
STS15
Spearman:67.97
STS16
Spearman:66.43
STSBenchmark
Spearman:62.57
SICKRelatedness
Spearman:64.05
Avg:63.64
********************************